In [41]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.read_csv('/usr/local/datasets/aac_shelter_outcomes.csv')




#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')
#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # Grazioso Salvare Logo image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
    html.Div('Jake Carter d5352982-cb86-11eb-b8bc-0242ac130003',),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    dcc.Link(html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))), href='www.https://www.snhu.edu/',),
   
    
    
    html.Hr(),
    
   
    
   html.Div(
    dcc.RadioItems(
    id='radio-items',
    options = [
        {'label': 'Water Rescue', 'value': 'water'},
        {'label': 'Mountain Rescue', 'value': 'mountain'},
        {'label': 'Disaster Rescue', 'value': 'disaster'},
        {'label': 'Reset', 'value': 'reset'}

        ],
    value = "",
    labelStyle={'display': 'inline-block'}
    ),
   
),
    
    
    
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        page_size=10,
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        
    ),
    html.Br(),
     html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    
    
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='map-id',
            className='col s12 m6',
            ),     
        html.Div(
        id='graph-id',
        className='col s12 m6',
        children = [
            dcc.Graph(id="graph")
        ])   
        ]), 
])
#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio-items', 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
        df = pd.DataFrame.from_dict(list(shelter.read({})))
        if (value == 'reset'): 
            
            df = pd.DataFrame.from_dict(list(shelter.read({})))
        elif (value == 'water'):                     
            
            df = pd.DataFrame.from_dict(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        elif (value == 'mountain'): 
            
            #df = pd.DataFrame.from_dict(list(shelter.read({"animal_type":"cat"})))
            df = pd.DataFrame.from_dict(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        
            
        elif (value == 'disaster'):
            
            df = pd.DataFrame.from_dict(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))
                               
                               
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)






#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    #Code for chart of your choice (e.g. pie chart) #
    df = pd.DataFrame.from_records(shelter.read({})) 
   
    return [
        
        dcc.Graph(
            figure = px.bar(df, x="animal_type", y="age_upon_outcome_in_weeks", color="animal_type")
        ) 
      
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app